In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import lightgbm as lgbm
import mlb
import os
BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
TRAIN_DIR = Path('../input/mlbpdeftrainupdate')

In [2]:
players = pd.read_csv(BASE_DIR / 'players.csv')

rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()

In [3]:
players 

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2056,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True
2057,672911,Jesus Cruz,1995-04-15,NaN,Salinas de Hidalgo,San Luis Potosi,Mexico,73,230,1,Pitcher,False
2058,676103,Damon Jones,1994-09-30,NaN,Twin Falls,ID,USA,77,233,1,Pitcher,True
2059,676755,Isaac Mattson,1995-07-14,NaN,Erie,PA,USA,74,205,1,Pitcher,True


In [4]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName','heightInches','weight']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob']
feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    'target1']

In [5]:
player_target_stats = pd.read_csv("../input/my-player-target-stat/player_target_stats.csv")
data_names=player_target_stats.columns.values.tolist()
data_names

['playerId',
 'target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob']

In [6]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])


# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

In [7]:
train_X = train[feature_cols]
train_y = train[['target1', 'target2', 'target3', 'target4']]

#_index = (train['date'] < 20210401)
_index = ((train['date'] > 20200529) & (train['date'] <= 20200801)) | ((train['date'] > 20190529) & (train['date'] <= 20190801)) | ((train['date'] > 20180529) & (train['date'] <= 20180801))
x_train1 = train_X.loc[~_index].reset_index(drop=True)
y_train1 = train_y.loc[~_index].reset_index(drop=True)
x_valid1 = train_X.loc[_index].reset_index(drop=True)
y_valid1 = train_y.loc[_index].reset_index(drop=True)

In [8]:
train_X = train[feature_cols2]
train_y = train[['target1', 'target2', 'target3', 'target4']]

#_index = (train['date'] < 20210401)
x_train2 = train_X.loc[~_index].reset_index(drop=True)
y_train2 = train_y.loc[~_index].reset_index(drop=True)
x_valid2 = train_X.loc[_index].reset_index(drop=True)
y_valid2 = train_y.loc[_index].reset_index(drop=True)

# LightGBM training

In [9]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score


params1 = {'objective':'mae',
           'reg_alpha': 0.14547461820098767, 
           'reg_lambda': 0.10185644384043743, 
           'n_estimators': 3333, 
           'learning_rate': 0.1046301304430488, 
           'num_leaves': 674, 
           'feature_fraction': 0.8101240539122566, 
           'bagging_fraction': 0.8884451442950513, 
           'bagging_freq': 8, 
           'min_child_samples': 51}

params2 = {
 'objective':'mae',
           'reg_alpha': 0.14947461820098767, 
           'reg_lambda': 0.10185644384043743, 
           'n_estimators': 3633, 
           'learning_rate': 0.08046301304430488, 
           'num_leaves': 64, 
           'feature_fraction': 0.9101240539122566, 
           'bagging_fraction': 0.9884451442950513, 
           'bagging_freq': 3, 
           'min_child_samples': 15
}

params4 = {'objective':'mae',
           'reg_alpha': 0.016468100279441976, 
           'reg_lambda': 0.09128335764019105, 
           'n_estimators': 9868, 
           'learning_rate': 0.10528150510326864, 
           'num_leaves': 157, 
           'feature_fraction': 0.5419185713426886, 
           'bagging_fraction': 0.2637405128936662, 
           'bagging_freq': 19, 
           'min_child_samples': 71}


params = {
 'objective':'mae',
#  'reg_alpha': 0.1,
#  'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 2021,
 "num_leaves": 127,
 'feature_fraction': 0.5419185713426886, 
 'bagging_fraction': 0.5637405128936662, 
 'bagging_freq': 15, 
}



# oof1, model1, score1 = fit_lgbm(
#     x_train1, y_train1['target1'],
#     x_valid1, y_valid1['target1'],
#     params1
#  )

# oof2, model2, score2 = fit_lgbm(
#     x_train2, y_train2['target2'],
#     x_valid2, y_valid2['target2'],
#     params2
# )

# oof3, model3, score3 = fit_lgbm(
#     x_train2, y_train2['target3'],
#     x_valid2, y_valid2['target3'],
#    params
# )

# oof4, model4, score4 = fit_lgbm(
#     x_train2, y_train2['target4'],
#     x_valid2, y_valid2['target4'],
#     params4
# )

# score = (score1+score2+score3+score4) / 4
# print(f'score: {score}')

# CatBoost

In [10]:
import pickle
from catboost import CatBoostRegressor

def fit_lgbm(x_train, y_train, x_valid, y_valid, target, params: dict=None, verbose=100):
    oof_pred_lgb = np.zeros(len(y_valid), dtype=np.float32)
    oof_pred_cat = np.zeros(len(y_valid), dtype=np.float32)
    
    if os.path.isfile(f'../input/mlb-lgbm-and-catboost-models/mymodel_lgb_{target}.pkl'):
        with open(f'../input/mlb-lgbm-and-catboost-models/mymodel_lgb_{target}.pkl', 'rb') as fin:
            model = pickle.load(fin)
    else:
    
        model = lgbm.LGBMRegressor(**params)
        model.fit(x_train, y_train, 
            eval_set=[(x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)

        with open(f'mymodel_lgb_{target}.pkl', 'wb') as handle:
            pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    oof_pred_lgb = model.predict(x_valid)
    score_lgb = mean_absolute_error(oof_pred_lgb, y_valid)
    print('mae:', score_lgb)
    
#     if os.path.isfile(f'../input/mlb-lgbm-and-catboost-models/mymodel_cb_{target}.pkl'):
#         with open(f'../input/mlb-lgbm-and-catboost-models/mymodel_cb_{target}.pkl', 'rb') as fin:
#             model_cb = pickle.load(fin)
#     else:
    
#         model_cb = CatBoostRegressor(
#                     n_estimators=2500,
#                     learning_rate=0.05,
#                     loss_function='MAE',
#                     eval_metric='MAE',
#                     max_bin=50,
#                     subsample=0.9,
#                     colsample_bylevel=0.6,
#                     verbose=100)

#         model_cb.fit(x_train, y_train, use_best_model=True,
#                          eval_set=(x_valid, y_valid),
#                          early_stopping_rounds=25)

#         with open(f'model_cb_{target}.pkl', 'wb') as handle:
#             pickle.dump(model_cb, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#     oof_pred_cat = model_cb.predict(x_valid)
#     score_cat = mean_absolute_error(oof_pred_cat, y_valid)
#     print('mae:', score_cat)
    
    return oof_pred_lgb, model, score_lgb


# training lightgbm
params = {
'boosting_type': 'gbdt',
'objective':'mae',
'subsample': 0.6,
'subsample_freq': 1,
'learning_rate': 0.03,
'num_leaves': 2**11-1,
'min_data_in_leaf': 2**12-1,
'feature_fraction': 0.6,
'max_bin': 100,
'n_estimators': 3000,
'boost_from_average': False,
"random_seed":2021,
}

oof_pred_lgb2, model_lgb2, score_lgb2 = fit_lgbm(
    x_train1, y_train1['target2'],
    x_valid1, y_valid1['target2'],
    2, params
)

oof_pred_lgb1, model_lgb1, score_lgb1 = fit_lgbm(
    x_train1, y_train1['target1'],
    x_valid1, y_valid1['target1'],
    1, params
)

oof_pred_lgb3, model_lgb3,score_lgb3 = fit_lgbm(
    x_train1, y_train1['target3'],
    x_valid1, y_valid1['target3'],
    3, params
)
oof_pred_lgb4, model_lgb4, score_lgb4= fit_lgbm(
    x_train1, y_train1['target4'],
    x_valid1, y_valid1['target4'],
    4, params
)

score = (score_lgb1+score_lgb2+score_lgb3+score_lgb4) / 4
print(f'LightGBM score: {score}')


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=4095, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4095
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 1.92822
[200]	valid_0's l1: 1.88537
[300]	valid_0's l1: 1.87438
[400]	valid_0's l1: 1.86841
[500]	valid_0's l1: 1.86399
[600]	valid_0's l1: 1.86074
[700]	valid_0's l1: 1.85813
[800]	valid_0's l1: 1.85598
[900]	valid_0's l1: 1.85446
[1000]	valid_0's l1: 1.85285
[1100]	valid_0's l1: 1.85131
[1200]	valid_0's l1: 1.85012
[1300]	valid_0's l1: 1.84903
[1400]	valid_0's l1: 1.8482
[1500]	valid_0's l1: 1.84754
[1600]	valid_0's l1: 1.84684
[1700]	valid_0's l1: 1.84616
[1800]	valid_0's l1: 1.84563
[1900]	valid_0's l1: 1.84505
[2000]	valid_0's l1: 1.84456
[2100]	valid_0's l1: 1.84395
[2200]	valid_0's l1: 1.84362
[2300]	valid_0's l1: 1.84327
[2400]	valid_0's l1: 1.84301
[2